In [134]:
import numpy as np
import scipy
import scipy.sparse as sps
import matplotlib.pyplot as plt
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import tensorflow as tf
from nltk.corpus import movie_reviews

In [135]:
num_of_features = 5000

## Loading the dataset

Dataset URL: https://www.kaggle.com/neiljs/all-shark-tank-us-pitches-deals

In [136]:
df = pd.read_csv('Sharktankpitchesdeals.csv')
df.head()

,Season_Epi_code,Pitched_Business_Identifier,Pitched_Business_Desc,Deal_Status,Deal_Shark
0,826,Bridal Buddy,a functional slip worn under a wedding gown th...,1,KOL+LG
1,826,Laid Brand,hair-care products made with pheromones . Laid...,0,NaN
2,826,Rocketbook,a notebook that can scan contents to cloud ser...,0,NaN
3,826,Wine & Design,painting classes with wine served . Wine & Des...,1,KOL
4,824,Peoples Design,a mixing bowl with a built-in scoop . Peoples ...,1,LG


In [137]:
for pitch in df.loc[:3, 'Pitched_Business_Desc']:
    print(pitch)
    print('-----------------------')

a functional slip worn under a wedding gown that allows the wearer to use the restroom on their own . Bridal Buddy is a lightweight slip worn under the gown that lets brides go to the bathroom while wearing it. When nature calls, the bride can bag up her bustle to safely relieve herself without making a mess.
-----------------------
hair-care products made with pheromones . Laid brand is a Â pheromone-enriched hair care product thatÂ enhances color, in addition to protecting and hydrating hair.Â The pheromones help girls â€œexude confidenceâ€ wherever she goes.
-----------------------
a notebook that can scan contents to cloud services via an app and can be erased by being microwaved . Rocketbook is an intelligent reusable notebook that allows users to write with a traditional pen and notebook. The notebook is erasable and reusable by allowing users to send notes to the cloud. Simply use a smartphone to send writings to the cloud and a microwave oven to erase for future use.
---------

In [138]:
corpus = [pitch for pitch in df.loc[:, 'Pitched_Business_Desc']]
corpus[:3]

['a functional slip worn under a wedding gown that allows the wearer to use the restroom on their own . Bridal Buddy is a lightweight slip worn under the gown that lets brides go to the bathroom while wearing it. When nature calls, the bride can bag up her bustle to safely relieve herself without making a mess.',
 'hair-care products made with pheromones . Laid brand is a Â\xa0pheromone-enriched hair care product thatÂ\xa0enhances color, in addition to protecting and hydrating hair.Â\xa0The pheromones help girls â€œexude confidenceâ€\x9d wherever she goes.',
 'a notebook that can scan contents to cloud services via an app and can be erased by being microwaved . Rocketbook is an intelligent reusable notebook that allows users to write with a traditional pen and notebook. The notebook is erasable and reusable by allowing users to send notes to the cloud. Simply use a smartphone to send writings to the cloud and a microwave oven to erase for future use.']

In [139]:
targets = [deal for deal in df.loc[:, 'Deal_Status']]
targets[:5]

[1, 0, 0, 1, 1]

In [140]:
set(targets)

{0, 1}

### Bag of words representation

In [141]:
count_vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=2, max_features=20)
bows = count_vectorizer.fit_transform(corpus)
pd.DataFrame(bows.toarray(), columns=count_vectorizer.get_feature_names()).head()

,allows,app,children,clothing,company,designed,device,free,high,itâ,kids,like,line,natural,offers,perfect,products,service,use,way
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [142]:
count_vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=2, max_features=num_of_features)
bows = count_vectorizer.fit_transform(corpus)
print("We have {} pitches.".format(bows.shape[0]))

We have 706 pitches.


### Producing training and test data

In [143]:
bows = bows.toarray().astype(np.float32)
targets = np.array(targets, dtype=np.float32)

## Better split

In [144]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(bows, targets, test_size=0.1, shuffle=True)

# Building the Network

In [145]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(20, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(20, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)
])

In [146]:
# we compile our neural network model
# we also have to choose an optimizer and a loss function
# for a binary classification task usually binary cross-entropy is fine
# we use accuracy as the metric
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [147]:
# training or in other words, fitting the model to the data
model.fit(X_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
18/18 [==============================] - 0s 9ms/step - loss: 0.6906 - accuracy: 0.5517 - val_loss: 0.6929 - val_accuracy: 0.4375
Epoch 2/10
18/18 [==============================] - 0s 4ms/step - loss: 0.6707 - accuracy: 0.5989 - val_loss: 0.6943 - val_accuracy: 0.4375
Epoch 3/10
18/18 [==============================] - 0s 3ms/step - loss: 0.6401 - accuracy: 0.6497 - val_loss: 0.6991 - val_accuracy: 0.4062
Epoch 4/10
18/18 [==============================] - 0s 3ms/step - loss: 0.5836 - accuracy: 0.7058 - val_loss: 0.7105 - val_accuracy: 0.4062
Epoch 5/10
18/18 [==============================] - 0s 3ms/step - loss: 0.4862 - accuracy: 0.8249 - val_loss: 0.7513 - val_accuracy: 0.4375
Epoch 6/10
18/18 [==============================] - 0s 3ms/step - loss: 0.3685 - accuracy: 0.9002 - val_loss: 0.8239 - val_accuracy: 0.5000
Epoch 7/10
18/18 [==============================] - 0s 3ms/step - loss: 0.2531 - accuracy: 0.9527 - val_loss: 0.9858 - val_accuracy: 0.5156
Epoch 8/10
18/18 [==

In [148]:
# Original [0.7539902329444885, 0.5188679099082947]
model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 2ms/step - loss: 1.5059 - accuracy: 0.5634


[1.505903720855713, 0.5633803009986877]

## Movie review dataset
Movie reviews - positive or negative

In [149]:
print(movie_reviews.raw('neg/cv000_29416.txt'))

plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
what's the deal ? 
watch the movie and " sorta " find out . . . 
critique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . 
which is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn't snag this one correctly . 
they seem to have taken this pretty neat concept , but executed it terribly . 
so what are the problems with the movie ? 
well , its main problem is that it's simply too jumbled . 
it starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience member , have no id

In [150]:
corpus, targets = zip(*[(movie_reviews.raw(fileid), category)
                         for category in movie_reviews.categories() for fileid in movie_reviews.fileids(category)])

In [151]:
targets[:10]

('neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg')

In [152]:
count_vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=2, max_features=num_of_features)
bows = count_vectorizer.fit_transform(corpus)
print("We have {} documents.".format(bows.shape[0]))

We have 2000 documents.


In [153]:
set(targets)

{'neg', 'pos'}

In [154]:
# convert targets to numbers
targets = np.array([0 if target == 'neg' else 1 for target in targets])
targets[:30]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [155]:
# we need to shuffle
perm = np.random.permutation(len(targets))
bows = bows[perm]
targets = targets[perm].astype(np.float32)

In [156]:
bows = bows.toarray().astype(np.float32)

In [157]:
X_train, X_test, y_train, y_test = train_test_split(bows, targets, test_size=0.1, shuffle=True)

In [158]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(20, activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
51/51 [==============================] - 0s 5ms/step - loss: 0.5547 - accuracy: 0.7309 - val_loss: 0.3653 - val_accuracy: 0.8778
Epoch 2/10
51/51 [==============================] - 0s 3ms/step - loss: 0.2335 - accuracy: 0.9377 - val_loss: 0.3025 - val_accuracy: 0.8778
Epoch 3/10
51/51 [==============================] - 0s 3ms/step - loss: 0.1071 - accuracy: 0.9889 - val_loss: 0.2761 - val_accuracy: 0.8833
Epoch 4/10
51/51 [==============================] - 0s 3ms/step - loss: 0.0506 - accuracy: 1.0000 - val_loss: 0.2826 - val_accuracy: 0.8833
Epoch 5/10
51/51 [==============================] - 0s 3ms/step - loss: 0.0279 - accuracy: 1.0000 - val_loss: 0.2861 - val_accuracy: 0.8889
Epoch 6/10
51/51 [==============================] - 0s 3ms/step - loss: 0.0169 - accuracy: 1.0000 - val_loss: 0.2970 - val_accuracy: 0.8889
Epoch 7/10
51/51 [==============================] - 0s 3ms/step - loss: 0.0114 - accuracy: 1.0000 - val_loss: 0.3054 - val_accuracy: 0.8889
Epoch 8/10
51/51 [==

In [159]:
# Original [0.4515152871608734, 0.8450000286102295]
model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 1ms/step - loss: 0.3863 - accuracy: 0.8950


[0.3863321542739868, 0.8949999809265137]